<a href="https://colab.research.google.com/github/MTirop/GitPractise/blob/main/Copy_of_Pandas_Data_Cleaning_Aggregation_PivotTables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pandas for Data Cleaning, Aggregations, and Pivot Tables

In this notebook, we'll cover some essential tasks in data analysis using **pandas**:
1. Data Cleaning
2. Aggregations
3. Pivot Tables

We'll use examples and simple datasets to demonstrate how to effectively manipulate and analyze data with pandas.


In [ ]:
import pandas as pd
import numpy as np



## 1. Data Cleaning

Data cleaning is the process of fixing or removing incorrect, corrupted, incorrectly formatted, duplicate, or incomplete data within a dataset.

### Common Data Cleaning Tasks:
- Missing values
- Duplicates
- Incorrect data types
- Inconsistent formatting
- Outliers


In [ ]:
# Create a sample "messy" dataset
data = {
    "Name": ["Alice", "Bob", None, "David ", "Eve", "Alice", "FRANK"],
    "Age": [25, np.nan, 22, 35, 29, 25, "50"],
    "City": ["New York", "Los Angeles", "new york", None, "Chicago", "New York", "BOSTON"],
    "Salary": ["$50,000", "$75,000", "$60,000", "90000", "$65,000", "$50,000", "$80000"],
    "Join_Date": ["2021-01-15", "01/15/2020", "2022/03/01", "2019-05-20", None, "2021-01-15", "20-07-2018"]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the original dataset
print("Original Dataset:")
df

NameError: name 'np' is not defined

In [ ]:
# Get a quick summary of the dataset
print("Basic DataFrame information:")
df.info()

In [ ]:
# Statistical summary (only works on numeric columns)
print("\nStatistical summary:")
df.describe(include='all')

In [ ]:

# Check for missing values
df.isnull()
#df.isnull().sum()

In [ ]:
# Approach 1: Remove rows with any missing values
# Drop rows with missing values
df_cleaned = df.dropna()
df_cleaned


In [ ]:
# Approach 2: Fill missing values with appropriate replacements
df_filled = df.copy()

# Fill missing names with "Unknown"
df_filled["Name"] = df_filled["Name"].fillna("Unknown")

# Fill missing ages with the mean age (first convert to numeric)
df_filled["Age"] = pd.to_numeric(df_filled["Age"], errors='coerce')
mean_age = df_filled["Age"].mean()
df_filled["Age"] = df_filled["Age"].fillna(mean_age)

# Fill missing cities with "Unknown"
df_filled["City"] = df_filled["City"].fillna("Unknown")

# Fill missing dates with the most frequent date
df_filled["Join_Date"] = df_filled["Join_Date"].fillna(df_filled["Join_Date"].mode()[0])

print("Dataset after filling missing values:")
df_filled

The errors='coerce' argument tells pandas to convert invalid parsing (e.g., non-numeric values like strings or symbols) into NaN (Not a Number) instead of throwing an error.

In [ ]:
# Check for duplicate rows
print(f"Number of duplicate rows: {df_filled.duplicated().sum()}")

# Show the duplicate rows
dupes = df_filled[df_filled.duplicated()]
print("\nDuplicate rows:")
print(dupes)

# Remove duplicates and keep the first occurrence
df_no_dupes = df_filled.drop_duplicates()

print(f"\nShape before removing duplicates: {df_filled.shape}")
print(f"Shape after removing duplicates: {df_no_dupes.shape}")
df_no_dupes

In [ ]:
# For demonstration: You can also remove duplicates based on specific columns
# For example, if we consider records with the same Name and Age as duplicates:
df_no_dupes_subset = df_filled.drop_duplicates(subset=["Join_Date"])
print(f"\nShape after removing duplicates based on Join_Date: {df_no_dupes_subset.shape}")
df_no_dupes_subset

In [ ]:
# Create a clean copy to work with
df_clean = df_no_dupes.copy()

# 1. Standardize text case in Name column (Title case)
df_clean["Name"] = df_clean["Name"].str.strip().str.title()

# 2. Standardize City names (Title case)
df_clean["City"] = df_clean["City"].str.strip().str.title()

# 3. Convert Age to integer
df_clean["Age"] = df_clean["Age"].astype(int)

# Display the updated DataFrame
print("After standardizing Names and Cities:")
df_clean

.str.strip() removes leading and trailing whitespaces
"  alice  " → "alice"

In [ ]:
# Convert Salary to numeric by removing $ and commas
df_clean["Salary"] = df_clean["Salary"].str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df_clean["Salary"] = pd.to_numeric(df_clean["Salary"])

# Check the data types
print("DataFrame data types after conversion:")
df_clean.dtypes

- str.replace() is used for string replacement.
- regex=False tells pandas to treat the 'dollar_sign' as a literal character, not a regular expression (where 'dollar_sign' means “end of string”).

Regex stands for Regular Expression.
It’s a powerful tool used for searching, matching, and manipulating text patterns — kind of like "smart find and replace" on steroids. e.g.

| Pattern | Meaning                                  | Example Match                                 |
|---------|------------------------------------------|-----------------------------------------------|
| `.`     | Any character (except newline)           | `a.b` → matches `acb`, `arb`, `a9b`           |
| `\d`    | Any digit (0-9)                          | `\d\d` → matches `23`, `45`                   |
| `\w`    | Any word character (a-z, A-Z, 0-9, _)    | `\w\w` → matches `ab`, `Z9`                   |
| `\s`    | Any whitespace (space, tab, etc)         |                                               |
| `*`     | 0 or more occurrences                    | `a*` → matches `""`, `a`, `aaa`               |
| `+`     | 1 or more occurrences                    | `a+` → matches `a`, `aaaa`                    |
| `?`     | 0 or 1 occurrence (optional)             | `a?b` → matches `b` or `ab`                   |
| `[]`    | Any one of the characters inside         | `[aeiou]` → matches any vowel like `a`, `e`   |
| `^`     | Start of string                          | `^Hello` → matches strings starting with "Hello" |
| `$`     | End of string                            | `end$` → matches strings ending in "end"      |


In [ ]:
# Rename columns for better clarity
df_clean = df_clean.rename(columns={
    "Name": "full_name",
    "Age": "age",
    "City": "city",
    "Salary": "annual_salary",
    "Join_Date": "joining_date"
})

# Display the cleaned dataset
print("Final cleaned dataset:")
df_clean


## 2. Aggregations

Aggregation involves performing operations like sum, mean, count, etc., on groups of data.

### Useful Functions:
- `groupby()`
- `agg()`
- `mean()`, `sum()`, `count()`, etc.


In [ ]:
# Create a dataset with more records for better aggregation examples
sales_data = {
    "store_id": ["A", "A", "A", "B", "B", "B", "C", "C", "C", "A", "B", "C"],
    "product_category": ["Electronics", "Clothing", "Grocery", "Electronics", "Clothing",
                          "Grocery", "Electronics", "Clothing", "Grocery", "Electronics", "Grocery", "Clothing"],
    "sale_date": pd.date_range(start="2023-01-01", periods=12, freq="D"),
    "sales_amount": [5200, 1500, 950, 4800, 1700, 1150, 3800, 1200, 880, 5500, 1250, 1350],
    "units_sold": [12, 30, 45, 10, 32, 55, 8, 24, 40, 13, 58, 27],
    "discount_applied": [True, False, False, True, False, True, False, True, False, True, True, False]
}

sales_df = pd.DataFrame(sales_data)
print("Sample sales data:")
sales_df

In [ ]:
# Basic statistics for the entire dataset
print("Overall statistics:")
sales_df.describe()

In [ ]:
# Sum of sales by store
print("\nTotal sales by store:")
sales_df.groupby("store_id")["sales_amount"].sum()



In [ ]:
# Average units sold by product category
print("\nAverage units sold by product category:")
sales_df.groupby("product_category")["units_sold"].mean()


In [ ]:
# Count of sales by store and whether discount was applied
print("\nCount of sales by store and discount status:")
sales_df.groupby(["store_id", "discount_applied"]).size()

In [ ]:
# Using lambda functions for quick custom aggregations
result = sales_df.groupby("product_category").agg({
    "sales_amount": [
        ("Total", "sum"),
        ("Average", "mean"),
        ("Range", lambda x: x.max() - x.min())
    ]
})

print("Using lambda functions in aggregations:")
result


## 3. Pivot Tables

- Pivot tables allow you to summarize data in a tabular format based on categorical data.
- Pivot tables reshape data to summarize information. They're similar to Excel pivot tables and help you see data from different angles.
### Syntax:
```python
pd.pivot_table(data, values, index, columns, aggfunc)
```


In [ ]:
# Create a basic pivot table: rows=store_id, columns=product_category, values=sales_amount
pivot1 = pd.pivot_table(
    sales_df,
    values="sales_amount",
    index="store_id",
    columns="product_category",
    aggfunc="sum"
)

print("Basic pivot table (sum of sales by store and product category):")
pivot1

In [ ]:
# Add row and column totals
pivot_with_margins = pd.pivot_table(
    sales_df,
    values="sales_amount",
    index="store_id",
    columns="product_category",
    aggfunc="sum",
    margins=True,
    margins_name="Total"
)

print("Pivot table with margins (totals):")
pivot_with_margins

- margins=True
Adds totals: both row totals and column totals.
- margins_name="Total"
Names the total row/column "Total" instead of the default "All".